In [1]:
import os
import re
import glob
import feather
import numpy as np
import pandas as pd
import unicodedata
import matplotlib.pyplot as plt
from itertools import product
from functools import partial
from pathlib import Path

# # dask
# import dask
# import dask.dataframe as dd
# from dask.distributed import Client

from src.clean_data import clean_rfc_fantasma, clean_sancionados, clean_rupc

# features generales
from src.features_general import (
    monto_por_unidad_compradora,
    proveedores_distintos,
    procedimientos_distintos,
    numero_de_contratos,
)

# features competencia
from src.features_competencia import (
    contratos_por_proveedor,
    porcentaje_procedimientos_por_tipo,
    porcentaje_monto_tipo_procedimiento,
    importe_promedio_por_contrato,
    calcular_IHH_ID_contratos,
    calcular_IHH_ID_monto,
)

# features transparencia
from src.features_transparencia import (
    porcentaje_procedimientos_presenciales,
    contratos_promedio_por_procedimimento,
    contratos_por_duracion,
    monto_por_duracion,
    promedio_datos_faltantes_poc_contrato,
)

# features anomalias
from src.features_anomalias import (
    interaccion_rfc_fantasma,
    interaccion_sancionados,
    porcentaje_contratos_por_convenio
)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline
%config IPCompleter.use_jedi = False

## Cargar tabla procedimientos

In [2]:
df_procedimientos = feather.read_dataframe(
    '../data/bases/procedimientos_all_2017_10_29.feather', nthreads=7)
print(df_procedimientos.shape)
df_procedimientos = df_procedimientos.loc[df_procedimientos.GOBIERNO == 'APF', :]
df_procedimientos = df_procedimientos.drop('GOBIERNO', axis=1)
df_procedimientos = df_procedimientos.assign(CLAVEUC=df_procedimientos.CLAVEUC_REAL)
df_procedimientos = df_procedimientos.drop('CLAVEUC_REAL', axis=1)
df_procedimientos = df_procedimientos.loc[~df_procedimientos.PROVEEDOR_CONTRATISTA.isnull()]
print(df_procedimientos.shape)
df_procedimientos = df_procedimientos.loc[df_procedimientos.CLAVEUC != 'MISSING']
print(df_procedimientos.shape)
df_procedimientos.head()

(1014554, 47)
(884997, 45)
(884960, 45)


,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,FECHA_ARCHIVO,IMPORTE_PESOS
70,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,nan,MULTISERVICIOS Y ASESORIAS DEL CARIBE,HABILITADO,None,319200.0,NaN,MXN,IA-009JZL032-N1-2012,MIXTA,INVITACION A CUANDO MENOS TRES,165782,SERVICIO DE LIMPIEZA CZM 2012,None,ADQUISICIONES,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,NaT,2011-12-16 00:00:00,2011-12-19,NaT,2012-01-01,2013-02-28,115807,SERVICIO DE LIMPIEZA CZM 2012,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,319200.0
71,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,nan,ALEJANDRO ALARCON JIMON,HABILITADO,None,26097.0,NaN,MXN,AA-009JZL032-N4-2012,PRESENCIAL,ADJUDICACION DIRECTA,165738,MANTENIMIENTO A EXTINTORES CZM,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,NO,2012-08-10 15:59:00,2012-08-10 16:05:00,NaT,2012-08-20,2012-08-26,2012-12-31,242370,SERVICIO DE MANTENIMIENTO A EXTINTORES AD CZM ...,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,26097.0
72,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,5578,TRANSPORTE ESPECIALIZADO DE PERSONAL NACIONAL,HABILITADO,MEDIANA,361200.0,NaN,MXN,IA-009JZL032-N2-2012,MIXTA,INVITACION A CUANDO MENOS TRES,148131,SERVICIO DE TRANSPORTE DE PERSONAL DE LA ESTAC...,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,2011-12-09 14:51:00,2011-12-16 14:00:00,2011-12-19,NaT,2012-01-01,2013-02-28,116667,SERVICIO DE TRANSPORTE DE PERSONAL CZM 2012,Z15122015 04 INVITACION A CUANDO MENOS TRES NA...,None,PoC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,361200.0
73,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MICRO,nan,COMERCIALIZADORA EL MAHARAJA,HABILITADO,None,334264.0,NaN,MXN,AA-009JZL032-N5-2011,MIXTA,ADJUDICACION DIRECTA,276283,SERVICIO DE COMEDOR DEL PERSONAL DE LA ESTACIO...,None,ADQUISICIONES,EXPIRADO,0.0,0.0,NACIONAL,0.0,SI,2011-12-28 12:45:00,2011-12-28 17:00:00,NaT,NaT,2012-01-01,2013-02-28,123933,SERVICIO DE COMEDOR AD CZM 2012,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,334264.0
74,AEROPUERTOS Y SERVICIOS AUXILIARES,ASA,ASA-ESTACION DE COMBUSTIBLES DEL AEROPUERTO DE...,009JZL032,JOSE LUIS ALMADA PENUNURI,MEDIANA,nan,COMERCIALIZADORA EL MAHARAJA,HABILITADO,None,285948.0,NaN,MXN,AA-009JZL032-N5-2011,MIXTA,ADJUDICACION DIRECTA,148083,SERVICIO DE COMEDOR PARA EL PERSONAL DE LA EST...,None,SERVICIOS,EXPIRADO,0.0,0.0,NACIONAL,NaN,NO,2011-12-28 12:45:00,2011-12-28 17:00:00,NaT,2011-12-30,2012-01-01,2012-12-31,123933,SERVICIO DE COMEDOR AD CZM 2012,Z15122015 07 ADJUDICACION DIRECTA NACIONAL ART...,None,UC,https://compranet.funcionpublica.gob.mx/esop/g...,Si,MX,NaN,None,NaN,2012,285948.0


## Cargar tabla scraper

In [3]:
cols_scraper = [
    'CODIGO_EXPEDIENTE',
    'archivo_anexos', 'archivo_apertura',
    'archivo_contrato', 'archivo_convocatoria',
    'archivo_fallo', 'archivo_junta',
    'numero_archivos','numero_convenios',
    # 'TIPO_PROCEDIMIENTO', 'TIPO_CONTRATACION', 'id_compranet',
]

df_scraper = pd.read_csv(
    '../data/bases/tabla_scraper_2017_10_29.csv',
    usecols=cols_scraper, dtype={'CODIGO_EXPEDIENTE': str, 'Year': str}
)
df_scraper = df_scraper.loc[:, cols_scraper]


## Join con procedimientos
cols_procs=[
    'CLAVEUC', 'NUMERO_PROCEDIMIENTO', 'CODIGO_EXPEDIENTE',
    'TIPO_PROCEDIMIENTO', 'TIPO_CONTRATACION'
]

df_contratos = (df_procedimientos.groupby(['CLAVEUC', 'NUMERO_PROCEDIMIENTO',
                                           'CODIGO_EXPEDIENTE', 'TIPO_PROCEDIMIENTO',
                                           'TIPO_CONTRATACION'], as_index=False).CODIGO_CONTRATO.count()
                                 .rename(columns={'CODIGO_CONTRATO': 'numero_contratos'}))

df_scraper = pd.merge(df_scraper, df_contratos, on='CODIGO_EXPEDIENTE', how='inner')

print(df_scraper.shape)
df_scraper.head(3)

(633906, 14)


,CODIGO_EXPEDIENTE,archivo_anexos,archivo_apertura,archivo_contrato,archivo_convocatoria,archivo_fallo,archivo_junta,numero_archivos,numero_convenios,CLAVEUC,NUMERO_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,TIPO_CONTRATACION,numero_contratos
0,599,0,1,0,0,1,1,58,0,018TOQ054,LO-018TOQ054-T2-2010,LICITACION PUBLICA,OBRA PUBLICA,1
1,1171,0,1,1,1,1,1,6,1,041A00001,LA-010A00001-N3-2011,LICITACION PUBLICA,SERVICIOS,1
2,1610,0,0,1,0,0,0,3,0,018TOQ999,IA-018TOQ999-I2-2011,INVITACION A CUANDO MENOS TRES,SERVICIOS,1


In [4]:
df_procs_adquisiciones = df_procedimientos.loc[df_procedimientos.TIPO_CONTRATACION == 'ADQUISICIONES']
df_procs_servicios = df_procedimientos.loc[df_procedimientos.TIPO_CONTRATACION == 'SERVICIOS']
df_procs_obra_publica = df_procedimientos.loc[df_procedimientos.TIPO_CONTRATACION == 'OBRA PUBLICA']
df_procs_arrendamientos = df_procedimientos.loc[df_procedimientos.TIPO_CONTRATACION == 'ARRENDAMIENTOS']

In [4]:
df_procedimientos.TIPO_PROCEDIMIENTO.value_counts()

ADQUISICIONES                       459383
SERVICIOS                           339353
OBRA PUBLICA                         59443
SERVICIOS RELACIONADOS CON LA OP     21208
ARRENDAMIENTOS                        5573
Name: TIPO_CONTRATACION, dtype: int64

### Clean rfc fantasma y sancionados

In [3]:
# RFC fantasma
df_fantasma = pd.read_csv(
    '../data/bases/RFC fantasma.csv', parse_dates=['Publicación página SAT definitivos'],
    usecols=['RFC', 'Nombre del Contribuyente', 'Publicación página SAT definitivos'],
    dtype={'RFC': str, 'Nombre del Contribuyente': str}, encoding='iso-8859-1', skiprows=2
)
df_fantasma = df_fantasma.rename(columns={'Nombre del Contribuyente': 'Nombre del Contribuyente'.upper()})

df_fantasma = df_fantasma.loc[df_fantasma.RFC != 'XXXXXXXXXXXX']
df_fantasma = clean_rfc_fantasma(df_fantasma)

# Proveedores sancionados
df_sancionados = pd.read_excel('../data/bases/SancionProveedoresContratistas.xls')
df_sancionados = clean_sancionados(df_sancionados)
# Hay un registro con expediente vacío, de la tabla de procedimientos se observa
# que los contratos los realizó en 2014
df_sancionados.loc[
    df_sancionados.PROVEEDOR_CONTRATISTA == 'ALBA MARIA DE LA ASUNCION HERRASTI FERNANDEZ', 'Expediente'
] = 'XXXX/2014'
df_sancionados = df_sancionados.assign(
    Year=df_sancionados.Expediente.map(
        lambda x: int(x.split('/')[1])
    )
)

# poner el valor predeterminado para fantasmo y sancionados
interaccion_rfc_fantasma = partial(interaccion_rfc_fantasma, df_rfc_fantasma=df_fantasma)
interaccion_sancionados = partial(interaccion_sancionados, df_sancionados=df_sancionados)

## Features scraper 

In [5]:
df_scraper_adquisiciones = df_scraper.loc[df_scraper.TIPO_CONTRATACION == 'ADQUISICIONES']
df_scraper_servicios = df_scraper.loc[df_scraper.TIPO_CONTRATACION == 'SERVICIOS']
# 'OBRA PUBLICA', 'ARRENDAMIENTOS', 'SERVICIOS RELACIONADOS CON LA OP'
print(df_scraper_adquisiciones.shape[0], df_scraper_servicios.shape[0])

278722 275608


In [51]:
df_test = promedio_convenios_por_proc(df_scraper_adquisiciones)
df_test = df_test.sort_values(df_test.columns[1], ascending=False)
print(df_test.shape)
df_test.head()

1502
(1502, 2)


,CLAVEUC,promedio_convenios
962,020VSS006,8.000000
315,006HBW005,8.000000
53,019GYR011,5.500000
209,018T4I008,5.000000
487,018TOQ934,4.333333


In [50]:
df_test.promedio_convenios.isnull().value_counts()

False    1502
Name: promedio_convenios, dtype: int64

In [42]:
# df_test = df_scraper.groupby(['CLAVEUC', 'numero_convenios'], as_index=False).CODIGO_EXPEDIENTE.count()
# df_test = df_test.rename(columns={'CODIGO_EXPEDIENTE': 'numero_procs'})
# df_test = df_test.assign(
#     convenio_por_procedimiento=df_test.numero_convenios.divide(df_test.numero_procs)
# )
# # df_test = df_test.pivot(index='CLAVEUC', columns='numero_convenios', values='CODIGO_EXPEDIENTE')
# # df_test = df_test.fillna(0)
# print(df_test.shape)
# df_test

In [31]:
df_scraper_adquisiciones.head()

,CODIGO_EXPEDIENTE,archivo_anexos,archivo_apertura,archivo_contrato,archivo_convocatoria,archivo_fallo,archivo_junta,numero_archivos,numero_convenios,CLAVEUC,NUMERO_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,TIPO_CONTRATACION,numero_contratos
8,2124,0,1,0,1,1,1,5,0,004I00001,LA-004I00001-N6-2011,LICITACION PUBLICA,ADQUISICIONES,2
10,2979,0,0,0,0,1,0,3,0,019GYR034,SA-019GYR034-N8-2011,ADJUDICACION DIRECTA,ADQUISICIONES,1
12,3376,0,0,0,0,0,0,0,0,018TOQ010,IA-018TOQ010-N11-2011,INVITACION A CUANDO MENOS TRES,ADQUISICIONES,1
16,3448,0,0,1,1,1,1,8,0,014000999,LA-014000999-N2-2011,LICITACION PUBLICA,ADQUISICIONES,1
22,2585,0,0,0,1,1,1,6,0,018TOQ015,LA-018TOQ015-N8-2011,LICITACION PUBLICA,ADQUISICIONES,1


In [25]:
# df_test.sort_values('pc_sin_junta', ascending=False)

In [5]:
# df_procedimientos.loc[df_procedimientos.CLAVEUC == '018T4I017']

In [6]:
# 'OBRA PUBLICA', 'ARRENDAMIENTOS', 'SERVICIOS RELACIONADOS CON LA OP', 'ADQUISICIONES'
# df_aux = df_scraper.loc[df_scraper.TIPO_CONTRATACION == 'SERVICIOS RELACIONADOS CON LA OP'].copy()
# df_test = df_aux.groupby(
#     ['CLAVEUC', 'numero_archivos'], as_index=False).CODIGO_EXPEDIENTE.count()
# df_test = df_test.pivot(index='CLAVEUC', columns='numero_archivos', values='CODIGO_EXPEDIENTE')
# df_test = df_test.fillna(0)

# df_test.head()

In [7]:
# df_scraper_servicios.head()

## feature de columnas faltantes

In [21]:
df_scraper.TIPO_CONTRATACION.value_counts(dropna=False)

ADQUISICIONES                       278722
SERVICIOS                           275608
OBRA PUBLICA                         55115
SERVICIOS RELACIONADOS CON LA OP     19892
ARRENDAMIENTOS                        4569
Name: TIPO_CONTRATACION, dtype: int64

In [7]:
df_test = df_feature.groupby('CLAVEUC', as_index=False).datos_faltantes.mean()

In [9]:
df_test.sort_values('datos_faltantes', ascending=False).head()

,CLAVEUC,datos_faltantes
738,015QEZ993,6.000000
655,014000969,5.750000
486,011000974,5.428571
151,008000978,5.287879
750,016000977,5.111111


In [37]:
# df_procedimientos.loc[df_procedimientos.CLAVEUC == '014000969'].TIPO_PROCEDIMIENTO.value_counts()

In [36]:
# df_procedimientos.loc[df_procedimientos.CLAVEUC == '014000969']

In [35]:
# df_adj = df_procedimientos.loc[df_procedimientos.TIPO_PROCEDIMIENTO == 'ADJUDICACION DIRECTA FEDERAL']
# df_adj.sample(n=100)

## Calcular features (Falta agregar a datos generales el scraper para poder filtrar)

In [4]:
funciones_procedimientos = {
    'general': [
        monto_por_unidad_compradora,
        proveedores_distintos,
        procedimientos_distintos,
        numero_de_contratos,
    ],
    'competencia': [
        contratos_por_proveedor,
        porcentaje_procedimientos_por_tipo,
        porcentaje_monto_tipo_procedimiento,
        importe_promedio_por_contrato,
        calcular_IHH_ID_contratos,
        calcular_IHH_ID_monto,
        
    ],
    'transparencia': [
        porcentaje_procedimientos_presenciales,
        contratos_promedio_por_procedimimento,
        contratos_por_duracion,
        monto_por_duracion,
        promedio_datos_faltantes_poc_contrato,
    ],
    'anomalias': [
        interaccion_rfc_fantasma,
        interaccion_sancionados,
        porcentaje_contratos_por_convenio,
    ]
}


funciones_scraper = {
    'general': [
        
    ],
}

conceptos = tuple(funciones_procedimientos.keys())

tipos_contratacion = (
    'ADQUISICIONES',
    'SERVICIOS',
    'OBRA PUBLICA',
    'ARRENDAMIENTOS',
    'SERVICIOS RELACIONADOS CON LA OP'
)

for concepto in conceptos:
    for tipo in tipos_contratacion:
        # sub-tablas
        df_procs_aux = df_procedimientos.loc[
            (df_procedimientos.TIPO_CONTRATACION == tipo)
        ]
        df_scraper_aux = df_scraper.loc[
            (df_scraper.TIPO_CONTRATACION == tipo)
        ]
        # se agarran las funciones para cada tabla
        functions_procs_in_concept = funciones_procedimientos[concepto]
        functions_scraper_in_concept = funciones_scraper[concepto]
        # Calculo de features
        df_features_procs = [
            function(df_procs_aux).set_index('CLAVEUC')
            for function in functions_procs_in_concept
        ]
        df_features_scraper = [
            function(df_scraper_aux).set_index('CLAVEUC')
            for function in functions_scraper_in_concept
        ]
        # join de features por tabla en concepto
        
        # Join de tablas en el concetp
        print(concepto, tipo)
        print([df.shape for df in df_features_procs])
        print('-' * 50)


general ADQUISICIONES
[(1529, 1), (1529, 1), (1529, 1), (1529, 1)]
--------------------------------------------------
general SERVICIOS
[(1640, 1), (1640, 1), (1640, 1), (1640, 1)]
--------------------------------------------------
general OBRA PUBLICA
[(926, 1), (926, 1), (926, 1), (926, 1)]
--------------------------------------------------
general ARRENDAMIENTOS
[(711, 1), (711, 1), (711, 1), (711, 1)]
--------------------------------------------------
general SERVICIOS RELACIONADOS CON LA OP
[(577, 1), (577, 1), (577, 1), (577, 1)]
--------------------------------------------------
competencia ADQUISICIONES
[(1529, 1), (1529, 6), (1529, 6), (1529, 1), (1529, 2), (1529, 2)]
--------------------------------------------------
competencia SERVICIOS
[(1640, 1), (1640, 6), (1640, 6), (1640, 1), (1640, 2), (1640, 2)]
--------------------------------------------------
competencia OBRA PUBLICA
[(926, 1), (926, 5), (926, 5), (926, 1), (926, 2), (926, 2)]
-------------------------------------

In [4]:
funciones_procedimientos = {
    'general': [
        monto_por_unidad_compradora,
        proveedores_distintos,
        procedimientos_distintos,
        numero_de_contratos,
    ],
    'competencia': [
        contratos_por_proveedor,
        porcentaje_procedimientos_por_tipo,
        porcentaje_monto_tipo_procedimiento,
        importe_promedio_por_contrato,
        calcular_IHH_ID_contratos,
        calcular_IHH_ID_monto,
        
    ],
    'transparencia': [
        porcentaje_procedimientos_presenciales,
        contratos_promedio_por_procedimimento,
        contratos_por_duracion,
        monto_por_duracion,
        promedio_datos_faltantes_poc_contrato,
    ],
    'anomalias': [
        interaccion_rfc_fantasma,
        interaccion_sancionados,
        porcentaje_contratos_por_convenio,
    ]
}


conceptos = tuple(funciones_procedimientos.keys())

tipos_contratacion = (
    'ADQUISICIONES',
    'SERVICIOS',
    'OBRA PUBLICA',
    'ARRENDAMIENTOS',
    'SERVICIOS RELACIONADOS CON LA OP'
)

for concepto in conceptos:
    for tipo in tipos_contratacion:
        # sub-tabla procedimientos
        df_procs_aux = df_procedimientos.loc[
            (df_procedimientos.TIPO_CONTRATACION == tipo)
        ]
        # sub-tabla scraper
        df_procs_aux = df_procedimientos.loc[
            (df_procedimientos.TIPO_CONTRATACION == tipo)
        ]
        functions_in_concept = funciones_procedimientos[concepto]
        df_features_procs = [
            function(df_procs_aux).set_index('CLAVEUC')
            for function in functions_in_concept
        ]
        # TODO: Por qué aquí no es outter?
        df_features = pd.concat(df_features, join='inner', axis=1)
        df_features = df_features.reset_index()
        # df_features = df_features.assign(nombre_uc=df_features.CLAVEUC.map(nombres_uc))
        # df_features = df_features.assign(dependencia=df_features.CLAVEUC.map(nombres_dep))
        file_path = os.path.join('../data/conceptos/', concepto, tipo, 'features.csv')
        df_features.to_csv(file_path, index=False, quoting=1, encoding='utf-8')
        print(file_path)
        print('-' * 50)

../data/conceptos/general/ADQUISICIONES/features.csv
--------------------------------------------------
../data/conceptos/general/SERVICIOS/features.csv
--------------------------------------------------
../data/conceptos/general/OBRA PUBLICA/features.csv
--------------------------------------------------
../data/conceptos/general/ARRENDAMIENTOS/features.csv
--------------------------------------------------
../data/conceptos/general/SERVICIOS RELACIONADOS CON LA OP/features.csv
--------------------------------------------------
../data/conceptos/competencia/ADQUISICIONES/features.csv
--------------------------------------------------
../data/conceptos/competencia/SERVICIOS/features.csv
--------------------------------------------------
../data/conceptos/competencia/OBRA PUBLICA/features.csv
--------------------------------------------------
../data/conceptos/competencia/ARRENDAMIENTOS/features.csv
--------------------------------------------------
../data/conceptos/competencia/SERVICI

In [10]:
df_test = pd.DataFrame(np.arange(0, 12).reshape(4, 3), columns=['uno', 'dos', 'tres'])
df_test = df_test.rename(columns={c: (c + '_algo') for c in df_test.columns})
df_test

,uno_algo,dos_algo,tres_algo
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11


## Análisis sancionados

In [5]:
df = pd.merge(
    df_procedimientos, df_sancionados,
    on='PROVEEDOR_CONTRATISTA', how='inner'
)
monto_por_contrato_fant = df.groupby(
    ['FECHA_ARCHIVO', 'DEPENDENCIA', 'CLAVEUC', 'PROVEEDOR_CONTRATISTA',
     'NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO'],
    as_index=False
).IMPORTE_PESOS.sum()

contratos = monto_por_contrato_fant.groupby(
    ['FECHA_ARCHIVO', 'DEPENDENCIA'], as_index=False).CODIGO_CONTRATO.count()
contratos = contratos.pivot(index='FECHA_ARCHIVO', columns='DEPENDENCIA', values='CODIGO_CONTRATO')
contratos = contratos.fillna(0)
contratos = contratos.astype(int)
top_dep_contratos = contratos.sum().sort_values(ascending=False).index[0:10]

In [ ]:
with plt.style.context('randy_olson_style'):
    import seaborn as sns
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20, 7))
    df_aux = contratos.loc[:, top_dep_contratos].T
    df_aux.columns.name = 'AÑO'
    sns.heatmap(df_aux, annot=True, cmap='magma', fmt='d')
    plt.tight_layout()

### Análisis de fantasma

In [7]:
# RFC fantasma
df_fantasma = pd.read_csv(
    '../data/processed/rfc_fantasmas.psv', sep='|', usecols=['RFC', 'NOMBRE DEL CONTRIBUYENTE'],
    dtype={'RFC': str, 'NOMBRE DEL CONTRIBUYENTE': str}
)
df_fantasma = df_fantasma.loc[df_fantasma.RFC != 'XXXXXXXXXXXX']
df_fantasma = clean_rfc_fantasma(df_fantasma)
df = pd.merge(
    df_procedimientos, df_fantasma,
    on='PROVEEDOR_CONTRATISTA', how='inner'
)
monto_por_contrato_fant = df.groupby(
    ['FECHA_ARCHIVO', 'DEPENDENCIA', 'CLAVEUC', 'PROVEEDOR_CONTRATISTA',
     'NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO'],
    as_index=False
).IMPORTE_PESOS.sum()

contratos = monto_por_contrato_fant.groupby(
    ['FECHA_ARCHIVO', 'DEPENDENCIA'], as_index=False).CODIGO_CONTRATO.count()
contratos = contratos.pivot(index='FECHA_ARCHIVO', columns='DEPENDENCIA', values='CODIGO_CONTRATO')
contratos = contratos.fillna(0)
contratos = contratos.astype(int)
top_dep_contratos = contratos.sum().sort_values(ascending=False).index[0:10]

In [ ]:
with plt.style.context('randy_olson_style'):
    import seaborn as sns
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(20, 7))
    df_aux = contratos.loc[:, top_dep_contratos].T
    df_aux.columns.name = 'AÑO'
    sns.heatmap(df_aux, annot=True, cmap='magma')
    plt.tight_layout()

In [1]:
# # RUPC
# df_rupc = pd.read_excel(
#     '../data/raw/RUPC_171011060240.xlsx',
#     dtype={'FOLIO_RUPC': str, 'RFC': str, 'CONTRATOS': int}
# )
# df_rupc = clean_rupc(df_rupc)
# 'JUAN RAMON GARCIA SANCHEZ'
# df_rupc.head()
# df_rupc.PROVEEDOR_CONTRATISTA.nunique(), df_rupc.FOLIO_RUPC.nunique()

# orden
* Fecha publicacion
* fecha apertura
* fecha fallo
* Fecha celebracion
* fecha inicio
* fecha fin

####  fecha apertura - fecha publicacion
#### fecha fallo - fecha_apertura
#### fecha celebracion - fecha fallo
####  fecha_fin - fecha_inico

## clean sancionados

In [4]:
def calcular_duracion(df, breakpoints=None, labels=None):
    """Cuenta el número de contratos que la unidad compradora tuvo
    en los intervalos de tiempo especificados"""
    if breakpoints is None:
        breakpoints = [0, 1, 5, 10, 20]
    if labels is None:
        labels = [
            'dias_sin_rango',
            'mismo_dia', 'uno_cinco_dias', 'cinco_diez_dias',
            'diez_veinte_dias', 'veinte_o_mas_dias'
        ]
    # TODO: verificar sizes de breapoints y labels
    monto_por_contrato = df.groupby(
        ['CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'NUMERO_PROCEDIMIENTO',
         'CODIGO_CONTRATO', 'FECHA_INICIO', 'FECHA_FIN'],
        as_index=False
    ).IMPORTE_PESOS.sum()
    delta_dias = (monto_por_contrato.FECHA_FIN - monto_por_contrato.FECHA_INICIO).dt.days
    monto_por_contrato = monto_por_contrato.assign(delta_dias=delta_dias)

    # adding labels
    monto_por_contrato = monto_por_contrato.assign(
        grupo_dias=monto_por_contrato.delta_dias.map(
            lambda d: labels[bisect.bisect(breakpoints, d)]
        )
    )
    return monto_por_contrato

df_test = calcular_duracion(df_procs_adquisiciones)
df_test.head()

,CLAVEUC,PROVEEDOR_CONTRATISTA,NUMERO_PROCEDIMIENTO,CODIGO_CONTRATO,FECHA_INICIO,FECHA_FIN,IMPORTE_PESOS,delta_dias,grupo_dias
0,002000997,ALBINO CHAVEZ DIAZ,IA-002000997-E19-2017,1369813,2017-01-04,2017-12-31,3189458.00,361,veinte_o_mas_dias
1,002000997,ALL DEPOT,IA-002000997-E29-2017,1404736,2017-04-26,2017-05-11,902454.53,15,diez_veinte_dias
2,002000997,ALL DEPOT,IA-002000997-E32-2017,1462608,2017-06-19,2017-12-31,487296.69,195,veinte_o_mas_dias
3,002000997,CAVANOVA,IA-002000997-E23-2017,1369840,2017-01-05,2017-12-31,769915.00,360,veinte_o_mas_dias
4,002000997,CEDAPSA,IA-002000997-E6-2016,1232347,2016-05-30,2016-06-21,463450.00,22,veinte_o_mas_dias


In [10]:
df_test.grupo_dias.value_counts(dropna=False)

veinte_o_mas_dias    252559
diez_veinte_dias     129254
cinco_diez_dias       49789
uno_cinco_dias        21750
mismo_dia              6091
Name: grupo_dias, dtype: int64

In [30]:
df_test = contratos_por_duracion(df_procs_adquisiciones)

print(df_test.shape[0], df_test.CLAVEUC.nunique())
df_test.head()

1529 1529


,CLAVEUC,contratos_mismo_dia,contratos_uno_cinco_dias,contratos_cinco_diez_dias,contratos_diez_veinte_dias,contratos_veinte_o_mas_dias
0,002000997,NaN,1.0,NaN,8.0,35.0
1,002000998,NaN,NaN,NaN,NaN,146.0
2,002000999,2.0,35.0,17.0,38.0,171.0
3,003000997,NaN,3.0,5.0,7.0,65.0
4,004000995,NaN,NaN,NaN,1.0,20.0


In [3]:
# df_test.sort_values(['NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO'])
# df_test.groupby('NUMERO_PROCEDIMIENTO').FECHA_INICIO.nunique().sort_values(ascending=False)

In [4]:
# df_test.loc[df_test.NUMERO_PROCEDIMIENTO == 'AA-012NAW001-N18-2015']

In [3]:
# # Porcentaje de procedimientos que empiezan y acaban el mismo día
# def diferencia_inicio_fin(df):
#     monto_por_contrato = df.groupby(
#         ['CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'NUMERO_PROCEDIMIENTO',
#          'CODIGO_CONTRATO', 'FECHA_INICIO', 'FECHA_FIN'],
#         as_index=False
#     ).IMPORTE_PESOS.sum()
#     delta_dias = (monto_por_contrato.FECHA_FIN - monto_por_contrato.FECHA_INICIO).dt.days
#     monto_por_contrato = monto_por_contrato.assign(delta_dias=delta_dias)
#     # adding labels
#     breakpoints = [0, 1, 5, 10, 20]
#     labels = [
#         'dias_sin_rango', 'mismo_dia', 'uno_cinco_dias',
#         'cinco_diez_dias', 'diez_veinte_dias', 'veinte_o_mas_dias'
#     ]
#     monto_por_contrato = monto_por_contrato.assign(
#         grupo_dias=monto_por_contrato.delta_dias.map(
#             lambda d: labels[bisect.bisect(breakpoints, d)]
#         )
#     )
#     return monto_por_contrato

# df_dias = diferencia_inicio_fin(df_procs_servicios)
# df_dias.head()

## 1 / Diferencia entre el valor promedio del importe del contrato Licitación vs Adj directa

## La UC trata con proveedores sancionados (dummy)

## Tratar con RFC fantasmas (Dummy)

In [9]:
# df_procs_obra_publica.loc[df_procs_obra_publica.CLAVEUC == '003000999']

In [8]:
# df_procs_servicios.loc[df_procs_servicios.CLAVEUC == '040100998']

In [3]:
# df_nombres_uc = df_procedimientos.loc[:, ['CLAVEUC', 'NOMBRE_DE_LA_UC']].drop_duplicates()
# nombres_uc = {row.CLAVEUC: row.NOMBRE_DE_LA_UC for row in df_nombres_uc.itertuples()}

# df_nombres_dep = df_procedimientos.loc[:, ['CLAVEUC', 'DEPENDENCIA']].drop_duplicates()
# nombres_dep = {row.CLAVEUC: row.DEPENDENCIA for row in df_nombres_dep.itertuples()}

# print(df_nombres.shape, df_nombres.CLAVEUC.nunique(), df_nombres.NOMBRE_DE_LA_UC.nunique())
# df_nombres.head()

In [15]:
# df_nombres.groupby('CLAVEUC').NOMBRE_DE_LA_UC.count().sort_values(ascending=False)

In [11]:
df_nombres.loc[df_nombres.CLAVEUC == '015000997']

,CLAVEUC,NOMBRE_DE_LA_UC
287579,015000997,SEDATU-SRA980428JG3
381081,015000997,SEDATU-SRA980428JG3 #015000997


## Calcular indicadores por tipo de contratacion

In [9]:
# df_procedimientos.loc[df_procedimientos.CLAVEUC == '006A00997'].PROVEEDOR_CONTRATISTA.value_counts()

## Calcular indicadores por año y tipo de contratacion

In [3]:
function_list = [
    # competencia
    contratos_por_proveedor,
    porcentaje_procedimientos_por_tipo,
    porcentaje_monto_tipo_procedimiento,
    importe_promedio_por_contrato,
    calcular_IHH,
    # transparencia
    porcentaje_procedimientos_presenciales
]


years = tuple(range(2012, 2018))
tipos_contratacion = (
    'ADQUISICIONES', 'SERVICIOS', 'OBRA PUBLICA', 'ARRENDAMIENTOS',
    'SERVICIOS RELACIONADOS CON LA OP'
)

for year, tipo in product(years, tipos_contratacion):
    
    df_aux = df_procedimientos.loc[
        (df_procedimientos.FECHA_INICIO.dt.year == year) &
        (df_procedimientos.TIPO_CONTRATACION == tipo)
    ]
    df_features = [function(df_aux).set_index('CLAVEUC') for function in function_list]
    df_features = pd.concat(df_features, join='inner', axis=1)
    df_features = df_features.reset_index()
    file_path = os.path.join('../data/features/', str(year), tipo, 'features.csv')
    df_features.to_csv(file_path, index=False, quoting=1, encoding='utf-8')
    print(file_path)

../data/features/2012/ADQUISICIONES/features.csv
../data/features/2012/SERVICIOS/features.csv
../data/features/2012/OBRA PUBLICA/features.csv
../data/features/2013/ADQUISICIONES/features.csv
../data/features/2013/SERVICIOS/features.csv
../data/features/2013/OBRA PUBLICA/features.csv
../data/features/2014/ADQUISICIONES/features.csv
../data/features/2014/SERVICIOS/features.csv
../data/features/2014/OBRA PUBLICA/features.csv
../data/features/2015/ADQUISICIONES/features.csv
../data/features/2015/SERVICIOS/features.csv
../data/features/2015/OBRA PUBLICA/features.csv
../data/features/2016/ADQUISICIONES/features.csv
../data/features/2016/SERVICIOS/features.csv
../data/features/2016/OBRA PUBLICA/features.csv
../data/features/2017/ADQUISICIONES/features.csv
../data/features/2017/SERVICIOS/features.csv
../data/features/2017/OBRA PUBLICA/features.csv


## Leer features

In [4]:
df_procedimientos.TIPO_CONTRATACION.value_counts(dropna=False)

ADQUISICIONES                       391062
SERVICIOS                           297925
OBRA PUBLICA                         54569
SERVICIOS RELACIONADOS CON LA OP     19438
ARRENDAMIENTOS                        4807
Name: TIPO_CONTRATACION, dtype: int64

In [9]:
{col: 'pc_contratos_' + col.replace(' ', '_').lower()  for col in df_procedimientos.TIPO_PROCEDIMIENTO.value_counts().index.values}

{'ADJUDICACION DIRECTA FEDERAL': 'pc_contratos_adjudicacion_directa_federal',
 'INVITACION A CUANDO MENOS 3 PERSONAS': 'pc_contratos_invitacion_a_cuando_menos_3_personas',
 'LICITACION PUBLICA': 'pc_contratos_licitacion_publica',
 'LICITACION PUBLICA CON OSD': 'pc_contratos_licitacion_publica_con_osd',
 'OTRO': 'pc_contratos_otro',
 'PROYECTO DE CONVOCATORIA': 'pc_contratos_proyecto_de_convocatoria'}

In [4]:
df_procs_adquisiciones = df_procedimientos.loc[df_procedimientos.TIPO_CONTRATACION == 'ADQUISICIONES']
df_procs_servicios = df_procedimientos.loc[df_procedimientos.TIPO_CONTRATACION == 'SERVICIOS']
df_procs_obra_publica = df_procedimientos.loc[df_procedimientos.TIPO_CONTRATACION == 'OBRA PUBLICA']


In [3]:
df_procedimientos.TIPO_CONTRATACION.value_counts(dropna=False)

ADQUISICIONES                       391062
SERVICIOS                           297925
OBRA PUBLICA                         54569
SERVICIOS RELACIONADOS CON LA OP     19438
ARRENDAMIENTOS                        4807
Name: TIPO_CONTRATACION, dtype: int64

In [10]:
df_procedimientos.ESTATUS_CONTRATO.value_counts(dropna=False)

EXPIRADO     700642
TERMINADO     41960
ACTIVO        25199
Name: ESTATUS_CONTRATO, dtype: int64

In [8]:
df_procedimientos.FORMA_PROCEDIMIENTO.value_counts(dropna=False)

Mixta                   263835
Presencial              229257
Electrónica             222010
NaN                      52658
Presencial (Estatal)        41
Name: FORMA_PROCEDIMIENTO, dtype: int64

In [9]:
monto_por_contrato = df_procs_adquisiciones.groupby(
    ['DEPENDENCIA', 'CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'NUMERO_PROCEDIMIENTO',
     'CODIGO_CONTRATO', 'FORMA_PROCEDIMIENTO'],
    as_index=False
).IMPORTE_PESOS.sum()
monto_por_contrato.head()

,DEPENDENCIA,CLAVEUC,PROVEEDOR_CONTRATISTA,NUMERO_PROCEDIMIENTO,CODIGO_CONTRATO,FORMA_PROCEDIMIENTO,IMPORTE_PESOS
0,ADMINISTRACION FEDERAL DE SERVICIOS EDUCATIVOS...,011C00999,2MAS DESARROLLO,AA-011C00999-E172-2016,1274658,Mixta,191689.41
1,ADMINISTRACION FEDERAL DE SERVICIOS EDUCATIVOS...,011C00999,2MAS DESARROLLO,AA-011C00999-E173-2016,1274614,Mixta,249135.98
2,ADMINISTRACION FEDERAL DE SERVICIOS EDUCATIVOS...,011C00999,2MAS DESARROLLO,AA-011C00999-E29-2016,1115557,Mixta,310332.04
3,ADMINISTRACION FEDERAL DE SERVICIOS EDUCATIVOS...,011C00999,2MAS DESARROLLO,AA-011C00999-E77-2016,1169944,Mixta,335215.13
4,ADMINISTRACION FEDERAL DE SERVICIOS EDUCATIVOS...,011C00999,2MAS DESARROLLO,AA-011C00999-E90-2016,1191267,Mixta,101735.38


# Features (ADQUISICIONES)

### Porcentaje de procesos presenciales

In [11]:
df_test = porcentaje_procedimientos_presenciales(df_procs_adquisiciones)
df_test.head()

,CLAVEUC,Electrónica,Mixta,Presencial,Presencial (Estatal)
0,002000997,20.000000,0.000000,80.000000,0.0
1,002000998,100.000000,0.000000,0.000000,0.0
2,002000999,1.612903,63.709677,34.677419,0.0
3,003000997,0.000000,4.000000,92.000000,4.0
4,004000995,0.000000,53.846154,46.153846,0.0


### Número de proveedores distintos por contrato

In [4]:
df_test = proveedores_por_contrato(df_procs_adquisiciones)
df_test.head()

,CLAVEUC,proveedores_por_contrato
0,002000997,0.900000
1,002000998,0.602564
2,002000999,0.698413
3,003000997,0.525000
4,004000995,0.857143


### Valor promedio del importe del contrato

In [5]:
df_test = importe_promedio_por_contrato(df_procs_adquisiciones)
df_test.head()

,CLAVEUC,monto_contrato_promedio
0,002000997,172016.703400
1,002000998,361292.915513
2,002000999,843075.567864
3,003000997,496076.516500
4,004000995,802756.780476


### Porcentaje de contrataciones (número y monto) por adjudicación directa e inv3

In [18]:
df_procedimientos.TIPO_PROCEDIMIENTO.value_counts(dropna=False)

ADJUDICACION DIRECTA FEDERAL            563561
LICITACION PUBLICA                      116820
INVITACION A CUANDO MENOS 3 PERSONAS     85263
OTRO                                      1471
LICITACION PUBLICA CON OSD                 388
PROYECTO DE CONVOCATORIA                   297
NaN                                          1
Name: TIPO_PROCEDIMIENTO, dtype: int64

In [6]:
df_test = porcentaje_procedimientos_por_tipo(df_procs_adquisiciones)
df_test.head()

,CLAVEUC,ADJUDICACION DIRECTA FEDERAL,INVITACION A CUANDO MENOS 3 PERSONAS,LICITACION PUBLICA,LICITACION PUBLICA CON OSD,OTRO,PROYECTO DE CONVOCATORIA
0,002000997,40.000000,60.000000,0.000000,0.0,0.0,0.0
1,002000998,0.000000,0.000000,100.000000,0.0,0.0,0.0
2,002000999,67.105263,21.710526,11.184211,0.0,0.0,0.0
3,003000997,16.000000,12.000000,72.000000,0.0,0.0,0.0
4,004000995,65.000000,35.000000,0.000000,0.0,0.0,0.0


In [7]:
df_test = porcentaje_monto_tipo_procedimiento(df_procs_adquisiciones)
df_test.head()

,CLAVEUC,ADJUDICACION DIRECTA FEDERAL,INVITACION A CUANDO MENOS 3 PERSONAS,LICITACION PUBLICA,LICITACION PUBLICA CON OSD,OTRO,PROYECTO DE CONVOCATORIA
0,002000997,29.430387,70.569613,0.000000,0.0,0.0,0.0
1,002000998,0.000000,0.000000,100.000000,0.0,0.0,0.0
2,002000999,52.351509,36.591765,11.056726,0.0,0.0,0.0
3,003000997,5.348814,2.439565,92.211621,0.0,0.0,0.0
4,004000995,58.155994,41.844006,0.000000,0.0,0.0,0.0


In [ ]:
monto_contrato = df_procs_adquisiciones.groupby(
    ['DEPENDENCIA', 'CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'NUMERO_PROCEDIMIENTO',
     'CODIGO_CONTRATO', 'TIPO_PROCEDIMIENTO'],
    as_index=False
).IMPORTE_PESOS.sum()

In [ ]:
## Porcentaje de procedimientos que empiezan y acaban el mismo día, etc

In [ ]:
## Número de convenios modificatorios

In [30]:
df_procedimientos.loc[df_procedimientos.CODIGO_EXPEDIENTE == '773759']

,DEPENDENCIA,SIGLAS,NOMBRE_DE_LA_UC,CLAVEUC,RESPONSABLE,ESTRATIFICACION_MUC,FOLIO_RUPC,PROVEEDOR_CONTRATISTA,ESTATUS_EMPRESA,ESTRATIFICACION_MPC,IMPORTE_CONTRATO,APORTACION_FEDERAL,MONEDA,NUMERO_PROCEDIMIENTO,FORMA_PROCEDIMIENTO,TIPO_PROCEDIMIENTO,CODIGO_CONTRATO,TITULO_CONTRATO,IDENTIFICADOR_CM,TIPO_CONTRATACION,ESTATUS_CONTRATO,COMPRA_CONSOLIDADA,PLURIANUAL,CARACTER,CONTRATO_MARCO,CONVENIO_MODIFICATORIO,PROC_F_PUBLICACION,FECHA_APERTURA_PROPOSICIONES,EXP_F_FALLO,FECHA_CELEBRACION,FECHA_INICIO,FECHA_FIN,CODIGO_EXPEDIENTE,TITULO_EXPEDIENTE,PLANTILLA_EXPEDIENTE,CLAVE_PROGRAMA,CUENTA_ADMINISTRADA_POR,ANUNCIO,ARCHIVADO,SIGLAS_PAIS,RAMO,ORGANISMO,C_EXTERNO,IMPORTE_PESOS


In [36]:
df_test = pd.merge(contratos_total, contratos_distintos, on=['CLAVEUC', 'NUMERO_PROCEDIMIENTO'], how='inner')
print(df_test.shape)

(260096, 4)


In [37]:
df_test.loc[df_test.conteo_contratos != df_test.nunique_contratos]

,CLAVEUC,NUMERO_PROCEDIMIENTO,conteo_contratos,nunique_contratos
32896,018TOQ937,AA-018TOQ937-N112-2013,2,1


In [7]:
# monto_contrato_adquisiciones.loc[monto_contrato_adquisiciones.NUMERO_PROCEDIMIENTO == 'AA-018TOQ937-N112-2013']

In [6]:
# df_procedimientos.loc[df_procedimientos.NUMERO_PROCEDIMIENTO == 'LA-019GYR047-N46-2014']

In [5]:
# df_procedimientos.loc[df_procedimientos.NUMERO_PROCEDIMIENTO == 'AA-018TOQ937-N112-2013']

In [ ]:
## Sacar un procedimiento con varios contratos

In [4]:
# monto_contrato_adquisiciones.loc[monto_contrato_adquisiciones.CLAVEUC == '006G2T002'].NUMERO_PROCEDIMIENTO.value_counts()

In [3]:
# monto_contrato_adquisiciones.groupby('NUMERO_PROCEDIMIENTO').CODIGO_CONTRATO.nunique().sort_values(ascending=False)

In [ ]:
# contratos_distintos = monto_contrato_adquisiciones.groupby(
#     ['CLAVEUC', 'NUMERO_PROCEDIMIENTO']).CODIGO_CONTRATO.nunique()
# contratos_distintos = contratos_distintos.reset_index().sort_values('CODIGO_CONTRATO', ascending=False)
# contratos_distintos = contratos_distintos.rename(columns={'CODIGO_CONTRATO': 'nunique_contratos'})
#

# # with plt.style.context('randy_olson_style'):
# #     df_feature.proveedores_por_contrato.plot.hist(50)
# # df_feature.sort_values('proveedores_por_contrato')
# # df_procedimientos.loc[df_procedimientos.CLAVEUC == '011MAX001']
# print(monto_contrato_adquisiciones.shape)
# print(
#     monto_contrato_adquisiciones.DEPENDENCIA.nunique(),
#     monto_contrato_adquisiciones.CLAVEUC.nunique(),
#     monto_contrato_adquisiciones.PROVEEDOR_CONTRATISTA.nunique(),
#     monto_contrato_adquisiciones.NUMERO_PROCEDIMIENTO.nunique(),
#     monto_contrato_adquisiciones.CODIGO_CONTRATO.nunique(),
# )
# monto_contrato_adquisiciones.head(2)

In [ ]:
# def calc_proveedores_por_contrato(df):
#     monto_contrato = df.groupby(
#         ['DEPENDENCIA', 'CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO'],
#         as_index=False
#     ).IMPORTE_PESOS.sum()
#     # ----------
#     pocs_distintos = monto_contrato_adquisiciones.groupby('CLAVEUC').PROVEEDOR_CONTRATISTA.nunique()
#     pocs_distintos = pocs_distintos.reset_index()
#     pocs_distintos = pocs_distintos.rename(columns={'PROVEEDOR_CONTRATISTA': 'proveedores_distintos'})

#     # -------------
#     contratos_total = monto_contrato_adquisiciones.groupby(
#         ['CLAVEUC', 'NUMERO_PROCEDIMIENTO']).CODIGO_CONTRATO.nunique()
#     contratos_total = contratos_total.reset_index()
#     contratos_total = contratos_total.rename(columns={'CODIGO_CONTRATO': 'conteo_contratos'})
#     contratos_total = contratos_total.groupby('CLAVEUC', as_index=False).conteo_contratos.sum()
#     # nunique_provs/num_contatros
#     df_feature = pd.merge(pocs_distintos, contratos_total, on='CLAVEUC', how='inner')
#     df_feature = df_feature.assign(
#         proveedores_por_contrato=df_feature.proveedores_distintos.divide(df_feature.conteo_contratos)
#     )
#     return df_feature.loc[:, ['CLAVEUC', 'proveedores_por_contrato']]


# def calcular_porcentaje_procedimientos_por_tipo(df):
#     monto_por_contrato = df.groupby(
#         ['DEPENDENCIA', 'CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'NUMERO_PROCEDIMIENTO',
#          'CODIGO_CONTRATO', 'TIPO_PROCEDIMIENTO'],
#         as_index=False
#     ).IMPORTE_PESOS.sum()
#     conteo_tipos = monto_por_contrato.groupby(
#         ['CLAVEUC', 'TIPO_PROCEDIMIENTO']
#     ).NUMERO_PROCEDIMIENTO.nunique().reset_index()
#     conteo_tipos = conteo_tipos.pivot(
#         index='CLAVEUC', columns='TIPO_PROCEDIMIENTO',
#         values='NUMERO_PROCEDIMIENTO'
#     ).fillna(0)
#     total_contratos = conteo_tipos.sum(axis=1)
#     conteo_tipos = conteo_tipos * 100
#     conteo_tipos = conteo_tipos.divide(total_contratos, axis='index')
#     conteo_tipos = conteo_tipos.reset_index()
#     conteo_tipos.columns.name = ''
#     return conteo_tipos


# def calcular_porcentaje_monto_tipo_procedimiento(df):
#     monto_por_contrato = df.groupby(
#         ['DEPENDENCIA', 'CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'NUMERO_PROCEDIMIENTO',
#          'CODIGO_CONTRATO', 'TIPO_PROCEDIMIENTO'],
#         as_index=False
#     ).IMPORTE_PESOS.sum()
#     monto_tipos = monto_contrato.groupby(
#         ['CLAVEUC', 'TIPO_PROCEDIMIENTO'], as_index=False
#     ).IMPORTE_PESOS.sum()
#     monto_tipos = monto_tipos.pivot(
#         index='CLAVEUC', columns='TIPO_PROCEDIMIENTO',
#         values='IMPORTE_PESOS'
#     ).fillna(0)
#     total_montos = monto_tipos.sum(axis=1)
#     monto_tipos = monto_tipos * 100
#     monto_tipos = monto_tipos.divide(total_montos, axis='index')
#     # TODO: cambiar el nombre de las columnas
#     monto_tipos = monto_tipos.reset_index()
#     monto_tipos.columns.name = ''
#     return monto_tipos

# df_procedimientos.sample(n=1000).to_csv(
#     '../data/processed/sample_procedimientos.psv', index=False, quoting=1, encoding='utf-8', sep='|'
# )

In [ ]:
# cols = [
#     'CLAVEUC',
#     'CODIGO_EXPEDIENTE',
#     'TIPO_CONTRATACION',
#     'TIPO_PROCEDIMIENTO',
#     'FORMA_PROCEDIMIENTO',
#     'CODIGO_CONTRATO',
#     'IMPORTE_PESOS',
#     'CONVENIO_MODIFICATORIO',
#     'CONTRATO_MARCO',
#     'COMPRA_CONSOLIDADA',
#     'PROVEEDOR_CONTRATISTA',
# ]

# df_procedimientos.loc[:, cols].to_csv(
#     '../data/results/procedimientos_reducida.psv', sep='|', index=False, quoting=1, encoding='utf-8')

In [ ]:
def contratos_por_duracion(df, breakpoints=None, labels=None):
    if breakpoints is None:
        breakpoints = [0, 1, 5, 10, 20]
    if labels is None:
        labels = [
            'dias_sin_rango',
            'mismo_dia', 'uno_cinco_dias', 'cinco_diez_dias',
            'diez_veinte_dias', 'veinte_o_mas_dias'
        ]
    # TODO: verificar sizes de breapoints y labels
    monto_por_contrato = df.groupby(
        ['CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'NUMERO_PROCEDIMIENTO',
         'CODIGO_CONTRATO', 'FECHA_INICIO', 'FECHA_FIN'],
        as_index=False
    ).IMPORTE_PESOS.sum()
    delta_dias = (monto_por_contrato.FECHA_FIN - monto_por_contrato.FECHA_INICIO).dt.days
    monto_por_contrato = monto_por_contrato.assign(delta_dias=delta_dias)
    
    # adding labels
    monto_por_contrato = monto_por_contrato.assign(
        grupo_dias=monto_por_contrato.delta_dias.map(
            lambda d: labels[bisect.bisect(breakpoints, d)]
        )
    )
    monto_por_contrato = monto_por_contrato.loc[
        :, ['CLAVEUC', 'delta_dias', 'grupo_dias', 'CODIGO_CONTRATO']
    ]
    conteo_contratos = monto_por_contrato.groupby(
        ['CLAVEUC', 'grupo_dias'], as_index=False
    ).CODIGO_CONTRATO.count()
    conteo_contratos = conteo_contratos.rename(
        columns={'CODIGO_CONTRATO': 'num_contratos'})
    conteo_contratos = conteo_contratos.pivot(
        index='CLAVEUC', columns='grupo_dias', values='num_contratos'
    )
    # ordenar salida
    conteo_contratos = conteo_contratos.loc[:, labels[1:]]
    
    conteo_contratos = conteo_contratos.rename(
        columns={c: 'contratos_' + c for c in conteo_contratos.columns}
    )
    conteo_contratos = conteo_contratos.reset_index()
    conteo_contratos.columns.name = ''
    conteo_contratos = conteo_contratos.fillna(0)
    return conteo_contratos

In [ ]:
def calcular_IHH_ID_contratos(df):
    monto_por_contrato = df.groupby(
        ['DEPENDENCIA', 'CLAVEUC', 'PROVEEDOR_CONTRATISTA',
         'NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO'],
        as_index=False
    ).IMPORTE_PESOS.sum()
    contratos_uc_poc = monto_por_contrato.groupby(
        ['CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'NUMERO_PROCEDIMIENTO'],
    ).CODIGO_CONTRATO.nunique()
    contratos_uc_poc = contratos_uc_poc.reset_index()
    contratos_uc_poc = contratos_uc_poc.groupby(
        ['CLAVEUC', 'PROVEEDOR_CONTRATISTA'], as_index=False
    ).CODIGO_CONTRATO.sum()
    contratos_uc = contratos_uc_poc.groupby(
        'CLAVEUC', as_index=False
    ).CODIGO_CONTRATO.sum()
    contratos_uc = contratos_uc.rename(
        columns={'CODIGO_CONTRATO': 'contratos_por_uc'}
    )
    contratos_uc_poc = pd.merge(
        contratos_uc_poc, contratos_uc, how='left', on='CLAVEUC'
    )
    contratos_uc_poc = contratos_uc_poc.assign(
        Share=(contratos_uc_poc.CODIGO_CONTRATO.divide(contratos_uc_poc.contratos_por_uc) * 100)
    )
    contratos_uc_poc = contratos_uc_poc.assign(
        IHH_contratos=contratos_uc_poc.Share**2
    )
    contratos_uc_poc = contratos_uc_poc.drop(
        ['contratos_por_uc', 'Share'], axis=1)
    # IHH por uc
    uc_IHH = contratos_uc_poc.groupby(
        'CLAVEUC', as_index=False).IHH_contratos.sum()
    print(uc_IHH.shape)
    uc_IHH = uc_IHH.rename(columns={'IHH_contratos': 'IHH_total_contratos'})
    # display(uc_IHH.head())
    
    # ID por uc
    contratos_uc_poc = pd.merge(
        contratos_uc_poc, uc_IHH, on='CLAVEUC', how='inner'
    )
    contratos_uc_poc = contratos_uc_poc.assign(
        ID_contratos=(
            contratos_uc_poc.IHH_contratos.divide(contratos_uc_poc.IHH_total_contratos)
        )
    )
    contratos_uc_poc = contratos_uc_poc.assign(
        ID_contratos=(contratos_uc_poc.ID_contratos * 100) ** 2
    )
    uc_ID = contratos_uc_poc.groupby('CLAVEUC', as_index=False).ID_contratos.sum()
    uc_ID = uc_ID.rename(columns={'ID_contratos': 'ID_total_contratos'})
    # final join
    df_feature = pd.merge(uc_IHH, uc_ID, on='CLAVEUC', how='inner')
    return df_feature

df_test = calcular_IHH_ID_contratos(df_procs_servicios)
df_test.head(20)

In [ ]:
# function_list = [
#     # general
#     monto_por_unidad_compradora,
#     # competencia
#     contratos_por_proveedor,
#     porcentaje_procedimientos_por_tipo,
#     porcentaje_monto_tipo_procedimiento,
#     importe_promedio_por_contrato,
#     calcular_IHH_monto,
#     calcular_IHH_contratos,
#     # transparencia
#     porcentaje_procedimientos_presenciales,
#     contratos_promedio_por_procedimimento,
#     contratos_por_duracion,
#     monto_por_duracion,
# ]

In [ ]:
def clean_rfc_fantasma(df_rfc_fantasma):
    df_rfc_fantasma = df_rfc_fantasma.rename(
        columns={'NOMBRE DEL CONTRIBUYENTE': 'PROVEEDOR_CONTRATISTA'})

    df = df.assign(
        PROVEEDOR_CONTRATISTA=(
            df.PROVEEDOR_CONTRATISTA.str.normalize('NFD').str.encode('ascii', 'ignore').str.decode('utf-8').str.upper()
        )
    )
    df = df.assign(
        PROVEEDOR_CONTRATISTA=df.PROVEEDOR_CONTRATISTA.str.replace('.', '')
    )
    df = df.assign(
        PROVEEDOR_CONTRATISTA=df.PROVEEDOR_CONTRATISTA.str.replace(',', '')
    )
    df = df.assign(
        PROVEEDOR_CONTRATISTA=df.PROVEEDOR_CONTRATISTA.str.strip()
    )
    df = df.assign(
        PROVEEDOR_CONTRATISTA=df.PROVEEDOR_CONTRATISTA.str.replace('"', '')
    )
    df = df.assign(
        PROVEEDOR_CONTRATISTA=df.PROVEEDOR_CONTRATISTA.str.replace("'", '')
    )
    df = df.assign(
        PROVEEDOR_CONTRATISTA=df.PROVEEDOR_CONTRATISTA.map(remove_double_white_space)
    )

    for regex in endings_regex_list:
        pattern = re.compile(regex)
        df = df.assign(
            PROVEEDOR_CONTRATISTA=df.PROVEEDOR_CONTRATISTA.map(
                lambda string: remove_pattern(string, pattern))
        )
    df = df.assign(PROVEEDOR_CONTRATISTA=df.PROVEEDOR_CONTRATISTA.str.strip())
    return df_rfc_fantasma
    
    

In [ ]:
# def rfc_fantasma(df_procs, df_rfc_fantasma):
#     df_feature = pd.DataFrame(
#         data=df_procs_adquisiciones.CLAVEUC.unique(),
#         columns=['CLAVEUC']
#     )
#     df = pd.merge(
#         df_procs, df_rfc_fantasma,
#         on='PROVEEDOR_CONTRATISTA', how='inner'
#     )
#     monto_por_contrato = df.groupby(
#         ['DEPENDENCIA', 'CLAVEUC', 'PROVEEDOR_CONTRATISTA',
#          'NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO'],
#         as_index=False
#     ).IMPORTE_PESOS.sum()
#     # número de proveedores fantasma por uc
#     pocs_distintos = monto_por_contrato.groupby('CLAVEUC').PROVEEDOR_CONTRATISTA.nunique()
#     pocs_distintos = pocs_distintos.reset_index()
#     pocs_distintos = pocs_distintos.rename(
#       columns={'PROVEEDOR_CONTRATISTA': 'num_proveedores_fantasma'})
    
#     # número de contratos con rfc fantasmas por uc
#     contratos_total = monto_por_contrato.groupby(
#         ['CLAVEUC', 'NUMERO_PROCEDIMIENTO']).CODIGO_CONTRATO.nunique()
#     contratos_total = contratos_total.reset_index()
#     contratos_total = contratos_total.rename(columns={'CODIGO_CONTRATO': 'contratos_con_fantasmas'})
#     contratos_total = contratos_total.groupby('CLAVEUC', as_index=False).contratos_con_fantasmas.sum()
    
#     # monto con rfc fantasmas por uc
#     monto_uc_contratos = monto_por_contrato.groupby(
#         ['CLAVEUC', 'NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO'], as_index=False
#     ).IMPORTE_PESOS.sum()
#     monto_uc_contratos = monto_uc_contratos.groupby('CLAVEUC', as_index=False).IMPORTE_PESOS.sum()
#     monto_uc_contratos = monto_uc_contratos.rename(columns={'IMPORTE_PESOS': 'monto_fantasma'})
    
#     # join the features
#     df_feature = pd.merge(df_feature, pocs_distintos, on='CLAVEUC', how='left')
#     df_feature = pd.merge(df_feature, contratos_total, on='CLAVEUC', how='left')
#     df_feature = pd.merge(df_feature, monto_uc_contratos, on='CLAVEUC', how='left')
#     df_feature = df_feature.fillna(0)
#     return df_feature

In [ ]:
df_codigos_repeditos = pd.read_csv(
    '../data/codigos_expediente_repetidos.csv', dtype=str, usecols=['CODIGO_EXPEDIENTE']
)
print(df_codigos_repeditos.shape, df_codigos_repeditos.CODIGO_EXPEDIENTE.nunique())
df_codigos_repeditos.head(2)

In [ ]:
# monto_por_contrato = df_procedimientos.groupby(
#     ['FECHA_ARCHIVO', 'CLAVEUC', 'PROVEEDOR_CONTRATISTA', 'NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO'],
#     as_index=False
# ).IMPORTE_PESOS.sum()

# monto_archivo = monto_por_contrato.groupby('FECHA_ARCHIVO').IMPORTE_PESOS.sum()
# monto_archivo

# contratos_total = monto_por_contrato.groupby(
#     ['FECHA_ARCHIVO', 'CLAVEUC', 'NUMERO_PROCEDIMIENTO']
# ).CODIGO_CONTRATO.nunique()
# contratos_total = contratos_total.reset_index()
# contratos_total = contratos_total.groupby('FECHA_ARCHIVO').NUMERO_PROCEDIMIENTO.nunique()
# contratos_total

In [ ]:
# cols_id = [
#     'DEPENDENCIA', 'CLAVEUC', 'PROVEEDOR_CONTRATISTA',
#     'NUMERO_PROCEDIMIENTO', 'CODIGO_CONTRATO'
# ]

# missing_cols = [
#     'EXP_F_FALLO', 'PROC_F_PUBLICACION',
#     'FECHA_APERTURA_PROPOSICIONES',
#     'FORMA_PROCEDIMIENTO', 'ANUNCIO', 'FECHA_CELEBRACION',
#     'FECHA_INICIO', 'FECHA_FIN'
# ]

# df_cols = df_procedimientos.loc[:, cols_id + missing_cols]
# monto_por_contrato = df_procedimientos.groupby(
#     cols_id, as_index=False).IMPORTE_PESOS.sum()

# df_feature = pd.merge(monto_por_contrato, df_cols, on=cols_id, how='inner')
# df_feature = df_feature.assign(
#     datos_faltantes=df_feature.loc[:, missing_cols].isnull().sum(axis=1)
# )
# df_feature = df_feature.groupby('CLAVEUC', as_index=False).datos_faltantes.mean()
# df_feature.head()